In [2]:
from oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json as js
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import pygsheets
import numpy as np
import lxml.html
import re


dic = {}
arr=["https://af.breezy.hr/", "https://gen-tech.breezy.hr/",
     "https://socialtech.breezy.hr/", "https://obrio-76be502da02f.breezy.hr/",
     "https://futurragroup.com/"]
arr_com = ["AppFlame", "Genesis", "SocialTech", "Obrio",  "Futurra"]
href_com = ["https://af.breezy.hr", "https://gen-tech.breezy.hr", "https://socialtech.breezy.hr", "https://obrio-76be502da02f.breezy.hr", ""]
pages = []
delete = []
new = []
k = 1
com = 0
for links in arr:
    url = links
    data  = requests.get(url).text
    soup = BeautifulSoup(data,"html5lib")
    for link in soup.find_all('a'):
        dic[k]=arr_com[com]+" "+str(link.get('href'))+" "+href_com[com]+str(link.get('href'))
        pages.append(href_com[com]+str(link.get('href')))
        k+=1
    com+=1


df=pd.DataFrame.from_dict(dic,orient='index')
df = df[0].str.split(" ", expand=True)
df.rename(columns = {0:'Company', 1:"Position", 2:"Link", 3:"Description"}, inplace = True)

for index, row in df.iterrows():
    if "analyst" not in row["Position"] :
        df.drop(index, inplace=True)

for str1 in pages:
    if str1.find("analyst") == -1:
        delete.append(str1)

for index, row in df.iterrows():
    j=0
    while row["Position"][j] != "-":
        j+=1
    row["Position"]=row["Position"][j+1::]


for i in pages:
    if i not in delete:
        new.append(i)

k=1
# descript = []
# for desc_links in new:
#     desc_url = desc_links
#     desc_data  = requests.get(desc_url).text
#     soup_desc = BeautifulSoup(desc_data,"html.parser")
#     desc = soup_desc.find_all("div", {"class": "description"})
#     descript.append(str(desc))
#     k+=1
#
#
# for i in range(len(descript)):
#     descript[i] = re.sub(r'<.*?>','',descript[i])
#
# df["Description"] = descript


gc = pygsheets.authorize(service_file="jsonFileFromGoogle.json")
sh = gc.open_by_key("1Fh4G25ZFa26cw_QfIV9x9IhjpW0FDMwj62cjMf_XTqA")
try:
    sh.add_worksheet("Sheet1")
except:
    pass
wks_write = sh.worksheet_by_title("Sheet1")
wks_write.clear('A1',None,'*')
wks_write.set_dataframe(df, (1,1), encoding='utf-8', fit=True)
wks_write.frozen_rows = 1

In [24]:
df.to_csv('data.csv', index = False)
